# Decision Tree

A decision tree is a popular machine-learning algorithm used for both classification and regression tasks. It models decisions and their possible consequences as a tree-like structure, where each internal node represents a "test" on an attribute, each branch represents the outcome of the test, and each leaf node represents a class label (in classification) or a continuous output (in regression).

### Key Components of a Decision Tree:
1. Nodes: There are two types of nodes in a decision tree i.e. Decision Nodes, Leaf Nodes
2. Branches: These are the links between nodes, representing the decision rules or outcomes of the tests performed at the decision nodes.

### How a Decision Tree Works:
- Splitting: Starting from the root of the tree, the data is split into subsets based on a feature that results in the highest "purity" (homogeneity) of the subsets. This is done using a criterion such as the gini index or the entropy in classification, and variance reduction in regression.
- Recursive Partitioning: This process is repeated recursively for each derived subset in a recursive manner. The recursion is completed when a stopping condition is met. Typical stopping conditions are a maximum tree depth, minimum number of samples in a node, or no further improvement in purity.
- Pruning: Sometimes, a fully grown tree is "pruned" by removing parts of the tree that provide little power in predicting target variables, to reduce overfitting and simplify the model.

**Creating a decision tree classifier from scratch** involves a few core components like calculating the best split based on a metric (like Gini impurity or entropy), recursively splitting the data, and then building the tree structure.

The Node class serves as the building block of the decision tree. Each node represents a decision point and can either be a decision node or a leaf node. 

To determine this each node is accounted with a feature index i.e., the index of the feature used for splitting the data at this node, threshold which is the value at which the split is made if the node is a decision node, left and right attributes that points to the left and right child of the node respectively and a value which is set only for leaf nodes and contains the class label that is most frequent in the data that reaches this leaf.

Gini impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. The formula for Gini impurity is:

                   J
    Gini(y) = 1 − ∑   Pi^2
                   i=1

where pi, is the proportion of class i elements in the set. 

Lower Gini impurity indicates a more "pure" node with predominant elements from a single class, which is the desired outcome in decision trees.

To find the best feature and threshold to split the data, we iterate through all features and all possible thresholds (by sorting unique values of a feature) and for each threshold, the data is divided into left and right subsets. Then the gini impurity is calculated for each subset, and a weighted average of these impurities is computed based on the size of the subsets. Finally, the split that results in the highest reduction in impurity (lowest weighted Gini impurity) is chosen as the best split.

The decision tree is created recursively by first checking whether the recursion should stop, which happens if all data at the node belong to one class or the maximum depth is reached, if the stopping condition is not met, it finds the best split to determine the best way to split the data. It then splits the data into left and right subsets and recursively calls itself to process these subsets. This recursion ends when it reaches a leaf node, where it returns a Node with the most frequent class in the subset.

For predictions, the tree is transversed for any input data starting from the root node, and based on the feature's index value and threshold it decides to move towards the left or right child. This process is repeated until a leaf node is reached, whose value attributes give the predicted class.

Below, I have provided an implementation using gini impurity as the metric to choose the splits. 

This example is a simplified version that assumes all features are numerical and the target is binary.

In [5]:
# Importing the required libraries
import pandas as pd
import numpy as np

# !pip install ucimlrepo
from ucimlrepo import fetch_ucirepo 

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [6]:
# Node Class represents a single decision node or a leaf in the decision tree.
class Node:
    # A Node in the Decision Tree.
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, *, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

In [7]:
# Computing the gini impurity of a node.
def gini_impurity(y):
    class_probs = np.bincount(y) / len(y)
    return 1 - np.sum(class_probs**2)

In [8]:
# A function to identify the optimal place to split the feature set to minimize impurity.
def best_split(X, y):
    m, n = X.shape
    best_gini = 1.0  # Starting with a worst-case scenario of maximum impurity
    best_idx, best_thr = None, None

    for index in range(n):  # Iterating over each feature
        thresholds, classes = zip(*sorted(zip(X[:, index], y)))  # Sort data along the feature
        
        # Ensuring that bincount arrays accommodate all class labels
        max_label = np.max(y) + 1
        left_classes = np.zeros(max_label, dtype=int)
        right_classes = np.bincount(classes, minlength=max_label)
        
        for i in range(1, m):  # Iterating over each threshold
            c = classes[i - 1]
            left_classes[c] += 1
            right_classes[c] -= 1
            
            if i < m and thresholds[i] != thresholds[i - 1]:  # Only calculate Gini if the threshold changes
                gini_left = gini_impurity(left_classes)
                gini_right = gini_impurity(right_classes)
                # Weighted Gini for left and right nodes
                gini = (i * gini_left + (m - i) * gini_right) / m
                
                if gini < best_gini:
                    best_gini = gini
                    best_idx, best_thr = index, (thresholds[i] + thresholds[i - 1]) / 2

    return best_idx, best_thr


In [9]:
# Build tree function recursively splits the data until the stopping criteria are met (e.g., max depth or minimal impurity gain).
# i.e. it builds the decision tree recursively.
def build_tree(X, y, depth=0, max_depth=10):
    num_samples_per_class = np.bincount(y)
    # Checking if only one class left or max depth reached
    if num_samples_per_class.size == 1 or depth == max_depth:
        # Returning a leaf node with class label
        leaf_value = np.argmax(num_samples_per_class)
        return Node(value=leaf_value)

    # Finding the best split
    feature_index, threshold = best_split(X, y)
    if feature_index is None:
        # Returning a leaf node if no split improves impurity
        leaf_value = np.argmax(num_samples_per_class)
        return Node(value=leaf_value)

    # Splitting the dataset
    left_idxs = X[:, feature_index] < threshold
    right_idxs = X[:, feature_index] >= threshold
    left_subtree = build_tree(X[left_idxs], y[left_idxs], depth + 1, max_depth)
    right_subtree = build_tree(X[right_idxs], y[right_idxs], depth + 1, max_depth)
    
    return Node(feature_index, threshold, left_subtree, right_subtree)

In [10]:
# Predicts the class label for a given sample using the tree
def predict(node, X):
    while node.value is None:
        try:
            feature_value = float(X[node.feature_index])
            threshold = float(node.threshold)
        except ValueError:
            print(f"Warning: Non-numeric data encountered: {X[node.feature_index]}")
            return None  # Or handle as needed
        
        if feature_value < threshold:
            node = node.left
        else:
            node = node.right
    return node.value


In [11]:
# Fitting builds the decision tree model by calling build_tree on the training data.
def fit(X, y):
# Ensuring that X and y are NumPy arrays of the right type
    y = np.array(y).flatten()  # Flattenning y to ensure it is 1D
    y = np.array(y, dtype=int)  # Ensuring y is of integer type
    X = np.array(X)
    return build_tree(X, y)

In [12]:
# Predicting labels for a dataset while ensuring data is passed correctly to predict.
def predict_labels(tree, X):
    return np.array([predict(tree, xi) if not any(isinstance(x, str) for x in xi) else None for xi in X.values])

# Example

In [14]:
# Loading the dataset either from the online repository directly or through the local storage


# # Fetching the data from the local storage
# data1 = pd.read_csv('winequality-red.csv', sep=';')
# data2 = pd.read_csv('winequality-white.csv', sep=';')

# # Combining the red and white wine data
# data = pd.concat([data1, data2])
# print(data.shape)

# # Determining the features and target varibale from 
# X = data[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphatesm', 'alcohol']]
# y = data['quality']


# fetching the dataset from the online UCI Machine Learning Repository 
wine_quality = fetch_ucirepo(id=186) 

# data (as pandas dataframes) 
X = wine_quality.data.features 
y = wine_quality.data.targets 

data = pd.merge(X, y, left_index=True, right_index=True)
print(data.shape)

(6497, 12)


In [15]:
# Checking for any missing or null values in the dataset
print(data.isnull().sum())

fixed_acidity           0
volatile_acidity        0
citric_acid             0
residual_sugar          0
chlorides               0
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64


In [16]:
# Looking at the overview of the dataset's columns
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         6497 non-null   float64
 1   volatile_acidity      6497 non-null   float64
 2   citric_acid           6497 non-null   float64
 3   residual_sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free_sulfur_dioxide   6497 non-null   float64
 6   total_sulfur_dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 609.2 KB
None


In [17]:
# Checking the summary of the dataset
data.describe()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712,0.873255
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000


In [18]:
# Looking at the uniques values in the 'quality' feature of the dataset
np.unique(y)

array([3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [19]:
# Converting the multi-class target into a binary classification problem
y = (y >= 7).astype(int)

In [20]:
# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [21]:
# Training the decision tree
tree = fit(X_train, y_train)

In [22]:
# Predicting on the test set
predictions = predict_labels(tree, X_test)

In [23]:
# Evaluating the model
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy of the decision tree on wine quality classification: {accuracy:.2f}")

Accuracy of the decision tree on wine quality classification: 0.81
